In [20]:
import numpy as np
import cv2 as cv
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from scipy.spatial.kdtree import KDTree


def parse_xml(xml_path):
    """parse xml file and calculate center point of each bounding box

    Args:
        xml_path (str): path of xml file

    Returns:
        str: dict variable includes center points and id of the bounding box
    """
    box_dict={}
    tree = ET.parse(xml_path)
    for obj in tree.findall("object"):
        cls = "tuft"
        # We include "difficult" samples in training.
        # Based on limited experiments, they don't hurt accuracy.
        # difficult = int(obj.find("difficult").text)
        # if difficult == 1:
        # continue

        bbox = obj.find("bndbox")
        bbox = [float(bbox.find(x).text) for x in ["xmin", "ymin", "xmax", "ymax"]]
        center_point=[(bbox[0]+bbox[2])/2,(bbox[1]+bbox[3])/2] # x, y
        box_dict.setdefault(obj.find("name").text, center_point)

    return box_dict


dataset="/media/ck/B6DAFDC2DAFD7F45/program/pyTuft/tiny-instance-segmentation/dataset/JPEGImages/"
refer_dir="/media/ck/B6DAFDC2DAFD7F45/program/pyTuft/2019/"
annotation_dir="/media/ck/B6DAFDC2DAFD7F45/program/pyTuft/tiny-instance-segmentation/dataset/Annotations/"

img1 = cv.imread(dataset+'DSC_2410.JPG',cv.IMREAD_GRAYSCALE)          # queryImage
img2 = cv.imread(dataset+'DSC_2411.JPG',cv.IMREAD_GRAYSCALE)          # trainImage
xml1=parse_xml(annotation_dir+'DSC_2410.xml')  # image1 bounding box
xml2=parse_xml(annotation_dir+'DSC_2411.xml')  # image2 bounding box

# Initiate SIFT detector
sift = cv.SIFT_create()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)
# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary
flann = cv.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)  #query, train, return 2 matches
# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches))]
# matched points list
pt1_list=[] # in img1
pt2_list=[] # in img2

In [21]:
# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches): # m: best match, n: second match
    if m.distance < 0.7*n.distance:
        matchesMask[i]=[1,0]
        pt1_list.append(list(kp1[m.queryIdx].pt))
        pt2_list.append(list(kp2[m.trainIdx].pt))

In [23]:
print(len(pt1_list),len(pt2_list))

291 291


In [27]:
pt1_array=np.array(pt1_list)
pt2_array=np.array(pt2_list)
tree1=KDTree(pt1_array)

xml2_array=np.array(list(xml2.values()))
tree2=KDTree(xml2_array)  # box tree

keys2=list(xml2.keys())

In [29]:
for key in xml1:
    dist1, ind=tree1.query(xml1[key],k=1) #find the closest matching point
    dist2, ind2=tree2.query(pt2_array[ind],k=1) #find the closest bounding box
    print("{}->{}".format(key,keys2[ind2]))

F4->F4
F1->F1
F2->F2
F3->F3
F5->F5
G1->G1
G2->F5
G3->G1
G4->G4
A5->A5
A1->A1
A2->A2
A3->A3
A4->A4
A6->A6
A7->A7
A8->A8
A9->A9
A10->A10
A11->A11
A12->A12
A13->A13
A14->G4
A15->A15
